In [ ]:
import pandas as pd  
import os
import warnings
warnings.filterwarnings('ignore')

# Merging 12 months of sales data into a single file

In [ ]:
df = pd.read_csv("./Sales_Data/Sales_April_2019.csv")

files = [file for file in os.listdir('./Sales_Data') ]

all_months_data = pd.DataFrame()

for file in files:
    df = pd.read_csv("./Sales_Data/"+file)
    all_months_data = pd.concat([all_months_data, df])
    
all_months_data.to_csv("all_data.csv", index= False)

# Read in updated DataFrame

In [ ]:
all_data = pd.read_csv("all_data.csv")
all_data.head() 

# Cleaning Up the data 

### Drop Row NaN

In [ ]:
nan_df = all_data[all_data.isna().any(axis=1)]
nan_df

In [ ]:
all_data = all_data.dropna(how='all')
all_data

### Finding 'Or' and deleting it   

In [ ]:
all_data = all_data[all_data['Order Date'].str[0:2] != 'Or']

### Convert coloums to the currect type 

In [ ]:
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each'] =pd.to_numeric(all_data['Price Each'])

# Augment data with additional Columns

## Task 2: Add Month Column  

In [ ]:
all_data ['Month'] = all_data['Order Date'].str[0:2]
all_data ['Month'] = all_data['Month'].astype('int32')
all_data

### Task3:  Add a sales Column 

In [ ]:
all_data ['Sales'] = all_data['Quantity Ordered']* all_data['Price Each']
all_data

## Add a city colunm

### lets us .apple() methord

In [ ]:
def get_city (address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1] 

#using F string  
all_data['City'] = all_data['Purchase Address'].apply(lambda x: f"{get_city(x)} ({get_state(x)})")


#all_data['city'] = all_data['Purchase Address'].apply(lambda x: get_city(x)+' ('+get_state(x)+')')

    
# all_data = all_data.drop(columns='Colunm')

# "without fution"
# all_data['Colunm'] = all_data['Purchase Address'].apply(lambda x: x.split(',')[1])

all_data.head()

### 1. what was the besh month for sales ? How much is earned that month?  

In [ ]:
results = all_data.groupby('Month').sum()['Sales']

In [ ]:
import matplotlib.pyplot as plt

months = range(1,13)

plt.bar(months, results)
plt.xticks(months)
plt.ylabel('Sales is US($)')
plt.xlabel('Months')
plt.show()

### 2. Which city has a highest number of sales? 

In [ ]:
results = all_data.groupby('City').sum()['Sales']
results

In [ ]:
import matplotlib.pyplot as plt
cities = [city for city, df in all_data.groupby('City')]

plt.bar(cities, results)
plt.xticks(cities, rotation='vertical', size =8)
plt.ylabel('Sales is US($)')
plt.xlabel('US City Name')
plt.show()

### 3. What time should we display advertisements to maximize the likelihood of customers buying products?

In [ ]:
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])

In [ ]:
all_data['Hour'] = all_data['Order Date'].dt.hour
all_data['Minute'] = all_data['Order Date'].dt.minute
all_data.head()

In [ ]:
hours = [hour for hour, df in all_data.groupby('Hour')]

plt.plot(hours, all_data.groupby(['Hour']).count())

all_data.groupby(['Hour']).count()
plt.xticks(hours)
plt.xlabel('Hours')
plt.ylabel('Number of Orders')
plt.grid()
plt.show()

###  4. What products are most often sold together?

In [ ]:
df = all_data[all_data['Order ID'].duplicated(keep = False)]

df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x:','.join(x))

df = df[['Order ID', 'Grouped']].drop_duplicates()

df.head()

In [ ]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key, value in count.most_common(5):
    print (key, value)

### 5. What product sold the most? Why do you think it sold the most?

In [ ]:
product_group = all_data.groupby('Product')

sum_by_product = product_group['Quantity Ordered'].sum()

In [ ]:
sum_by_product

In [ ]:
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])

In [ ]:
product_group = all_data.groupby('Product')
quantity_ordered = product_group['Quantity Ordered'].sum()

plt.bar(quantity_ordered.index, quantity_ordered.values)
plt.xlabel('Product')
plt.ylabel('Quantity Ordered')
plt.title('Total Quantity Ordered for Each Product')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'], errors='coerce')
price = all_data.groupby('Product')['Price Each'].mean()
print(price)

In [ ]:
# Assuming 'Price Each' column needs to be converted to numeric
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'], errors='coerce')

# Assuming 'Product' and 'Quantity Ordered' columns are present in your DataFrame
quantity_ordered = all_data.groupby('Product')['Quantity Ordered'].sum()
prices = all_data.groupby('Product')['Price Each'].mean()

# Plotting
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(quantity_ordered.index, quantity_ordered, color='y')
ax2.plot(prices.index, prices, color='b')

ax1.set_xlabel('Product Name')
ax1.set_ylabel('Quantity Ordered', color='y')
ax2.set_ylabel('Price ($)', color='b')
ax1.set_xticklabels(quantity_ordered.index, rotation='vertical', size=8)

plt.show()
